Import needed libraries

In [1]:
import os
import sys
from pathlib import Path
import pandas as pd
from pydantic import BaseModel
from typing import Optional,Literal

sys.path.append("..")

from utils.llm_client import LLMClient
from utils.prompts import render
from utils.llm_client import LLMClient
from utils.login_utils import log_llm_call
from utils.router import pick_model, should_use_reasoning_model


In [2]:
districts = Literal[
    "Ampara", "Anuradhapura", "Badulla", "Batticaloa", "Colombo",
    "Galle", "Gampaha", "Hambantota", "Jaffna", "Kalutara",
    "Kandy", "Kegalle", "Kilinochchi", "Kurunegala", "Mannar",
    "Matale", "Matara", "Monaragala", "Mullaitivu", "Nuwara Eliya",
    "Polonnaruwa", "Puttalam", "Ratnapura", "Trincomalee", "Vavuniya","Not Mentioned"
]

In [20]:
class CrisisEvent(BaseModel):
    district : districts = 'Not Mentioned'
    flood_level_meters: Optional[float] = None
    victim_count: int = 0
    main_need: str = None
    status: Literal["Critical", "Warning", "Stable"] = None

In [21]:
json_schema = {
    "district": "One of ['Not Mentioned', 'Ampara', 'Anuradhapura', 'Badulla', 'Batticaloa', 'Colombo', 'Galle', 'Gampaha', 'Hambantota', 'Jaffna', 'Kalutara', 'Kandy', 'Kegalle', 'Kilinochchi', 'Kurunegala', 'Mannar', 'Matale', 'Matara', 'Monaragala', 'Mullaitivu', 'Nuwara Eliya', 'Polonnaruwa', 'Puttalam', 'Ratnapura', 'Trincomalee', 'Vavuniya']",
    "flood_level_meters": "Optional[float], water level in meters if mentioned if not Null",
    "victim_count": "integer, number of people affected (default 0 if unknown)",
    "main_need": "Check for the need No need if not mentioned",
    "status": "One of ['Critical', 'Warning', 'Stable']"
}

In [23]:
model = pick_model("groq", "general")
llm_client = LLMClient("groq", model)

with open("../data/News Feed.txt", "r", encoding="utf-8") as f:
    texts = [line.strip() for line in f if line.strip()]

crisis_events = []

for text in texts:
    prompt_text, spec = render(
        "json_extract.v1",
        schema=json_schema,
        text=text
    )

    messages = [{"role": "user", "content": prompt_text}]
    response = llm_client.chat(messages, temperature=0.0)

    response_text = response["text"]

    try:
        crisis_event = CrisisEvent.model_validate_json(response_text)
        crisis_events.append(crisis_event)
    except Exception as e:
        print(f"Skipping invalid response: {response_text}\nError: {e}")

df = pd.DataFrame([event.model_dump() for event in crisis_events])

df.to_excel("../output/flood_report.xlsx", index=False)